In [ ]:
# %matplotlib notebook

from wild_visual_navigation import WVN_ROOT_DIR
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

folder, model_name = "scene_adaptation", False
# folder, model_name = "loss_with_tmp", False
# p = os.path.join(WVN_ROOT_DIR, f"scripts/ablations/{folder}_ablation/{folder}_ablation_test_results.pkl")
ws = os.environ["ENV_WORKSTATION_NAME"]
p = os.path.join(WVN_ROOT_DIR, f"results/ablations/{folder}_ablation_{ws}/{folder}_ablation_test_results.pkl")

with open(p, "rb") as f:
    res = pickle.load(f)

import scipy
import pandas as pd
import pickle
import copy

data = {
    "run_nr": [],
    "cfg_name": [],
    "train_scene": [],
    "test_scene": [],
    "test_auroc_gt": [],
    "test_auroc_prop": [],
    "test_auroc_anomaly_prop": [],
    "test_auroc_anomaly_gt": [],
}

for train_scene_name, scene_data in res.items():
    for config_path, config_data in scene_data.items():
        if model_name:
            name = config_path
        else:
            # its a path
            name = config_path.split("/")[-1][:-5]
        for run_nr, run_data in config_data.items():
            for test_scene_name, test_scene_data in run_data.items():
                data["cfg_name"].append(name)
                data["run_nr"].append(run_nr)
                data["train_scene"].append(train_scene_name)
                data["test_scene"].append(test_scene_name)
                data["test_auroc_gt"].append(test_scene_data["test_auroc_gt_image"])
                data["test_auroc_prop"].append(test_scene_data["test_auroc_proprioceptive_image"])
                data["test_auroc_anomaly_prop"].append(
                    test_scene_data.get("test_auroc_anomaly_proprioceptive_image", 0)
                )
                data["test_auroc_anomaly_gt"].append(test_scene_data.get("test_auroc_anomaly_gt_image", 0))

df = pd.DataFrame.from_dict(data)


data_new = {
    "cfg_name": [],
    "train_scene": [],
    "test_scene": [],
    "gt_mean": [],
    "prop_mean": [],
    "ano_prop_mean": [],
    "ano_gt_mean": [],
    "gt_std": [],
    "prop_std": [],
    "ano_prop_std": [],
    "ano_gt_std": [],
}
keys = []
for i in range(len(df)):
    n = data["cfg_name"][i]
    tr = data["train_scene"][i]
    te = data["test_scene"][i]

    a = df[(df["train_scene"] == tr) * (df["test_scene"] == te) * (df["cfg_name"] == n)]
    k = n + tr + te
    if k in keys:
        continue
    keys.append(k)
    a["test_auroc_gt"].mean()
    a["test_auroc_gt"].std()
    data_new["cfg_name"].append(n)
    data_new["train_scene"].append(tr)
    data_new["test_scene"].append(te)
    data_new["gt_mean"].append(a["test_auroc_gt"].mean())
    data_new["prop_mean"].append(a["test_auroc_prop"].mean())
    data_new["ano_prop_mean"].append(a["test_auroc_anomaly_prop"].mean())
    data_new["ano_gt_mean"].append(a["test_auroc_anomaly_gt"].mean())
    data_new["gt_std"].append(a["test_auroc_gt"].std())
    data_new["prop_std"].append(a["test_auroc_prop"].std())
    data_new["ano_prop_std"].append(a["test_auroc_anomaly_prop"].std())
    data_new["ano_gt_std"].append(a["test_auroc_anomaly_gt"].std())


df = pd.DataFrame.from_dict(data_new)
df = df.round(decimals=4)
df[df.select_dtypes(include=["number"]).columns] = df[df.select_dtypes(include=["number"]).columns] * 100
df = df.sort_values("train_scene", ascending=False)